In [ ]:
pip install tqdm

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms, models
from torch.utils.data import DataLoader, random_split
import os
from tqdm import tqdm

In [ ]:
from google.colab import drive
drive.mount('/content/drive')
SAMPLE_SIZE = 0.30
os.makedirs('/content/drive/MyDrive/checkpoints', exist_ok=True)

Mounted at /content/drive


In [ ]:
# Set device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Path to dataset
data_dir = '/content/drive/MyDrive/DL_Data/nature_12K/inaturalist_12K'

# Transforms
image_size = 224  # ResNet50 expects 224x224
transform = transforms.Compose([
    transforms.Resize((image_size, image_size)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406],
                         std=[0.229, 0.224, 0.225])
])

# Load full training dataset (we'll split it later)
full_train_dataset = datasets.ImageFolder(os.path.join(data_dir, 'train'), transform=transform)

# Split into 80% train and 20% validation
train_size = int(0.8 * len(full_train_dataset))
val_size = len(full_train_dataset) - train_size
train_dataset, val_dataset = random_split(full_train_dataset, [train_size, val_size])

# Load test dataset
test_dataset = datasets.ImageFolder(os.path.join(data_dir, 'val'), transform=transform)

# DataLoaders
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=32, shuffle=False)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)

# Load pre-trained ResNet50 model
model = models.resnet50(pretrained=True)

# Freeze all layers
for param in model.parameters():
    param.requires_grad = False

# Replace final layer to match 10-class iNaturalist dataset
num_ftrs = model.fc.in_features
model.fc = nn.Linear(num_ftrs, 10)

# Move model to GPU/CPU
model = model.to(device)

# Only train the final layer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.fc.parameters(), lr=0.001)

# Train the model
num_epochs = 10
for epoch in range(num_epochs):
    model.train()
    total_loss, correct, total = 0.0, 0, 0

    print(f"\nEpoch [{epoch+1}/{num_epochs}]")
    train_bar = tqdm(train_loader, desc="Training", leave=False)
    for images, labels in train_bar:
        images, labels = images.to(device), labels.to(device)

        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        total_loss += loss.item() * images.size(0)
        _, predicted = torch.max(outputs, 1)
        correct += (predicted == labels).sum().item()
        total += labels.size(0)

        train_acc = correct / total
        train_loss = total_loss / total
        train_bar.set_postfix(loss=train_loss, acc=train_acc)

    print(f"Train Loss: {train_loss:.4f}, Accuracy: {train_acc:.4f}")

    # Validation
    model.eval()
    val_correct, val_total = 0, 0
    val_bar = tqdm(val_loader, desc="Validating", leave=False)
    with torch.no_grad():
        for images, labels in val_bar:
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            _, predicted = torch.max(outputs, 1)
            val_total += labels.size(0)
            val_correct += (predicted == labels).sum().item()

            val_acc = val_correct / val_total
            val_bar.set_postfix(acc=val_acc)

    print(f"Validation Accuracy: {val_acc:.4f}")

# Final test accuracy
model.eval()
test_correct, test_total = 0, 0
test_bar = tqdm(test_loader, desc="Testing")
with torch.no_grad():
    for images, labels in test_bar:
        images, labels = images.to(device), labels.to(device)
        outputs = model(images)
        _, predicted = torch.max(outputs, 1)
        test_total += labels.size(0)
        test_correct += (predicted == labels).sum().item()

print(f"\nFinal Test Accuracy: {test_correct / test_total:.4f}")


FileNotFoundError: [Errno 2] No such file or directory: '/content/drive/MyDrive/DL_Data/nature_12K/inaturalist_12K/train'